In [1]:
import pandas as pd 
import numpy as np 
import os 
import pickle
import urllib
import requests
import json


In [2]:
import matplotlib.pyplot as plt
import gensim
import numpy as np
import spacy

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim

import os, re, operator, warnings
warnings.filterwarnings('ignore')  # Let's not pay heed to them right now
%matplotlib inline

In [3]:
def url_cleaning(df):
    """
    
    """
    
    df = (df.assign(url_new = lambda x : x.url.str.split(":",expand=True)[1]) #remove http and https 
            .sort_values(by='pub_date', ascending=False, na_position='last')
            .drop_duplicates(subset='url_new', keep='first'))
    return df 

In [4]:
def vg_plus_cleaning(df):
    """
    clean vgs + related articles (subscription related articles)
    """
    df = df.drop(df.index[df.text_len<30])
    return df

In [5]:
def read_vg_file(file):
    """
    
    """
    #df = pd.read_json(json.dumps(pickle.load( open( data_folder+name, "rb" ) )))
    
    #data = pd.DataFrame()
    #pd.DataFrame.from_dict(pickle.load(open("../spiders/data/vg/20.03.01_18:56:02.pickle","rb")))
    df = (pd.DataFrame.from_dict(pickle.load(open(file,'rb')))
            .replace('None', np.nan)
            .assign(text_len= lambda x: x.text.str.len())
            .assign(pub_date=lambda x: pd.to_datetime(x['pub_date']))
            .dropna(axis=0,subset=['text'])
            .sort_values(by='pub_date',na_position='last')
            .pipe(url_cleaning)
            .pipe(vg_plus_cleaning)#test[test.x > 0].index
         )
    return df


In [6]:
folder = "../spiders/data/vg/"
name1 = "20.03.01_21:00:01.pickle"
data = read_vg_file(folder+name1)

orginalt: 34931 etter: 34468

In [69]:
def get_vg_data(folder):
    """
    
    """
    files = os.listdir(folder)
    df = pd.DataFrame(columns = ['url','text','title','pub_date'])
    for file in files:
        #read, sanity check, sort on date, remove duplicates
        # merge with exicting, sort on date, remove duplicates 
        pass
    

In [8]:
nlp = spacy.load("nb_core_news_sm")

name = '20.01.23 23:19:58.pickle'
name = "20.03.01_18:56:02.pickle"
data_folder = "../spiders/data/vg/"
with open(data_folder+name, 'rb') as handle:
    b = pickle.load(handle)

In [9]:
data.sample(3)

,text,title,url,pub_date,text_len,url_new
558,– Young Bloods skal knuses. Vi må ta bakmennen...,Frp-leder: Vil ha oppholdsforbud og besøksforb...,https://www.vg.no/nyheter/innenriks/i/VR3GBr/f...,2018-08-14 11:24:51,2304.0,//www.vg.no/nyheter/innenriks/i/VR3GBr/frp-led...
16849,Senterpartiet akter å fremme mistillitsforslag...,Mistillitsforslag mot miljøministeren: Hvem bl...,https://www.vg.no/nyheter/meninger/i/mmK2q/mis...,2017-04-21 10:47:10,4445.0,//www.vg.no/nyheter/meninger/i/mmK2q/mistillit...
9952,Barnelegen Abdulrahim Essa (46) viser oss rund...,Gazas nyfødte dør av medisinmangel – VG,https://www.vg.no/nyheter/utenriks/i/zLP3G1/ga...,2018-04-21 14:59:02,2438.0,//www.vg.no/nyheter/utenriks/i/zLP3G1/gazas-ny...


In [10]:
url_stop_ord = "https://gist.githubusercontent.com/kmelve/8869818/raw/407980a1d89a385283d738e1545ab1b2014e87be/stoppord_no"
response = requests.get(url_stop_ord,)
my_stop_words = response.content.decode("utf-8") .split("\n")

In [11]:
for stopword in my_stop_words:
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True

In [14]:
data['text'][:20].astype('unicode').apply(nlp)

0        (Ullevål, sykehus, skjerpet, søndag, sine, kar...
35033    (Folkehelseinstituttet, opplyser, at, tolv, av...
35011    (Folkehelseinstituttet, opplyser, at, tolv, av...
35012    ( , Myndighetene, i, Seoul, har, anmeldt, Lee,...
35009    (Nå, krever, Norges, største, parti, at, det, ...
30       (I, fjor, fortalte, politifolk, i, Kripos, til...
13       (En, konsekvens, av, Det, hører, med, i, dette...
26       (I, går, kveld, Andersen, tiltrådte, som, stat...
29       (–, Vi, har, sett, dette, tydelig, de, siste, ...
28       (Du, vinker, meg, til, side, i, korridoren, .,...
11       ( , Wold, bor, og, jobber, i, Istanbul, ,, men...
6        (17, personer, har, fått, påvist, coronaviruse...
14       (Det, kommer, frem, i, en, oppdatering, fra, O...
34       (Det, kommer, frem, i, en, oppdatering, fra, O...
51       (Det, kommer, frem, i, en, oppdatering, fra, O...
23       (Marcello, Longhi, er, prest, i, en, av, mange...
3        (–, Alle, de, 15, har, relasjon, til, utbredel.

In [16]:
%%time

data['text'][:200].apply(nlp)

CPU times: user 11.6 s, sys: 1.19 s, total: 12.7 s
Wall time: 12.9 s


0        (Ullevål, sykehus, skjerpet, søndag, sine, kar...
35033    (Folkehelseinstituttet, opplyser, at, tolv, av...
35011    (Folkehelseinstituttet, opplyser, at, tolv, av...
35012    ( , Myndighetene, i, Seoul, har, anmeldt, Lee,...
35009    (Nå, krever, Norges, største, parti, at, det, ...
                               ...                        
36       (Lørdag, uttalte, tidligere, kulturminister, O...
31128    (Vel, ,, ikke, helt, ., Med, mindre, de, noe, ...
31126    (Mens, britenes, statsminister, Boris, Johnson...
31141    (Mens, britenes, statsminister, Boris, Johnson...
31168    (Mens, britenes, statsminister, Boris, Johnson...
Name: text, Length: 200, dtype: object

In [17]:
data.text

0        Ullevål sykehus skjerpet søndag sine karantene...
35033    Folkehelseinstituttet opplyser at tolv av tilf...
35011    Folkehelseinstituttet opplyser at tolv av tilf...
35012     Myndighetene i Seoul har anmeldt Lee Man-hee,...
35009    Nå krever Norges største parti at det nedsette...
                               ...                        
5384     Lykkelig over at katten ikke ser ut til å ha n...
925      Jo, det er Tom Hanks. Juryen har talt.Det var ...
30096    Ifølge TV 2 har Nygaard nylig mottatt et brev ...
4562     Norske TV-seere har kunnet følge Dawson, spilt...
5366     Hun merket ikke at en av husets katter hadde l...
Name: text, Length: 34468, dtype: object

In [22]:
data.shape

(34468, 6)

In [24]:
data.shape[0]/1000 * 1.05

36.19140000000001

In [21]:
%%time
data['text'].sample(1000).apply(nlp)

CPU times: user 57.6 s, sys: 7.14 s, total: 1min 4s
Wall time: 1min 5s


12516    (Vertslandet, Norge, oppjusterer, også, antall...
32670    (Det, er, solgt, cirka, 22.000, billetter, til...
4385     (Det, statlige, nyhetsbyrået, SANA, skriver, a...
20452    (–, Jeg, synes, det, er, gøy, å, prøve, litt, ...
9073     (Belgia, hadde, lenge, problemer, med, å, bryt...
                               ...                        
10623    (Kilder, Ifølge, CNN, hevder, Cohen, å, ha, væ...
19644    (–, Vi, kunne, lent, oss, tilbake, og, gått, p...
27711    (–, Jeg, er, totalt, forandret, siden, sist, ....
16828    (Til, mandag, er, det, skolestart, over, hele,...
362      (I, en, Twitter, -, melding, natt, til, tirsda...
Name: text, Length: 1000, dtype: object

In [27]:
data.head()#.index

,text,title,url,pub_date,text_len,url_new
0,Ullevål sykehus skjerpet søndag sine karantene...,Ullevål sykehus med nye karanteneregler – VG,https://www.vg.no/nyheter/innenriks/i/8mK43W/u...,2020-03-01 19:18:53,1519.0,//www.vg.no/nyheter/innenriks/i/8mK43W/ullevaa...
35033,Folkehelseinstituttet opplyser at tolv av tilf...,Nye tall: Totalt 19 corona-smittede i Norge – VG,https://www.vg.no/nyheter/innenriks/i/K375oe/n...,2020-03-01 19:11:54,1767.0,//www.vg.no/nyheter/innenriks/i/K375oe/nye-tal...
35011,Folkehelseinstituttet opplyser at tolv av tilf...,Nye tall: Totalt 19 corona-smittede i Norge – VG,https://www.vg.no/nyheter/innenriks/i/K375oe/n...,2020-03-01 19:11:54,1767.0,//www.vg.no/nyheter/innenriks/i/K375oe/nye-tal...
35012,"Myndighetene i Seoul har anmeldt Lee Man-hee,...",Sektleder i Sør-Korea pågrepet etter corona-ut...,https://www.vg.no/nyheter/utenriks/i/OpJxLq/se...,2020-03-01 17:51:02,1029.0,//www.vg.no/nyheter/utenriks/i/OpJxLq/sektlede...
35009,Nå krever Norges største parti at det nedsette...,Vil stemme for full lønnsstopp for Erna Solber...,https://www.vg.no/nyheter/innenriks/i/e84pBl/v...,2020-03-01 16:21:31,3789.0,//www.vg.no/nyheter/innenriks/i/e84pBl/vil-ste...


In [28]:
data['processes_text'] = pd.Series(np.nan, index=data.index)

In [53]:
chunk

array([ 1268,  6994, 21251, ..., 30096,  4562,  5366])

In [54]:
%%time

for i, chunk in enumerate(np.array_split(data.index.values, 30)):
    print(f'fold {i}')
    data['processes_text'].loc[chunk] = data['text'].loc[chunk].apply(nlp)

fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9
fold 10
fold 11
fold 12
fold 13
fold 14
fold 15
fold 16
fold 17
fold 18
fold 19
fold 20
fold 21
fold 22
fold 23
fold 24
fold 25
fold 26
fold 27
fold 28
fold 29
CPU times: user 34min 42s, sys: 9min 41s, total: 44min 24s
Wall time: 46min 37s


In [ ]:
1586464

In [59]:
data.memory_usage(index=True).sum()/10**6

3.516672

In [12]:
%%time
for indecies in data.index
    data['processes_text'] = data['text'].apply(nlp)



KeyboardInterrupt: 

In [ ]:
def parallelize_dataframe(df, func, n_cores=4):
    """
    https://towardsdatascience.com/make-your-own-super-pandas-using-multiproc-1c04f41944a1
    """
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
%%time
lass = parallelize_dataframe(df.text,nlp)#train = parallelize_dataframe(train_df, add_features)


In [61]:
# we add some words to the stop word list
texts, article = [], []
for pd_article in data.processes_text:
    for w in pd_article: 
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            article.append(w.lemma_)
    texts.append(article)
    article = []



In [62]:
bigram = gensim.models.Phrases(texts)
texts = [bigram[line] for line in texts]


In [63]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

## LSI

In [68]:
lsimodel = LsiModel(corpus=corpus, num_topics=20, id2word=dictionary)
lsimodel.show_topics(num_topics=10)  # Showing only the top 5 topics


[(0,
  '0.292*"VG" + 0.253*" " + 0.158*"møte" + 0.155*"parti" + 0.154*"fortelle" + 0.124*"skje" + 0.123*"vite" + 0.107*"barn" + 0.106*"Trump" + 0.105*"snakke"'),
 (1,
  '0.393*"parti" + -0.332*"spiller" + -0.279*"spille" + 0.268*"KrF" + -0.190*"Solskjær" + 0.183*" " + -0.175*"klubbe" + 0.171*"Frp" + -0.154*"sesong" + -0.134*"kamp"'),
 (2,
  '-0.558*"Trump" + 0.372*"parti" + 0.245*"KrF" + 0.196*"spiller" + 0.173*"spille" + -0.148*"barn" + 0.145*"Frp" + -0.135*"president" + 0.118*"Solskjær" + -0.110*" "'),
 (3,
  '0.711*" " + -0.451*"Trump" + -0.230*"parti" + 0.151*"barn" + 0.128*"VG" + -0.112*"KrF" + -0.111*"møte" + -0.108*"president" + 0.092*"skole" + -0.075*"valg"'),
 (4,
  '-0.572*" " + 0.345*"barn" + -0.325*"Trump" + 0.297*"VG" + 0.228*"fortelle" + -0.147*"spiller" + -0.117*"spille" + -0.103*"Solskjær" + 0.097*"skole" + -0.097*"parti"'),
 (5,
  '0.603*"VG" + -0.522*"barn" + -0.184*"skole" + -0.144*"  " + 0.138*"møte" + -0.121*"parti" + 0.109*"fortelle" + -0.106*"små" + -0.093*"forel

## HDP

In [65]:
hdpmodel = HdpModel(corpus=corpus, id2word=dictionary)
hdpmodel.show_topics()


[(0,
  '0.006*VG + 0.005*  + 0.003*fortelle + 0.003*møte + 0.003*parti + 0.003*skje + 0.002*Trump + 0.002*vite + 0.002*legge + 0.002*dag + 0.002*uke + 0.002*små + 0.002*snakke + 0.002*jo + 0.002*følge + 0.002*barn + 0.002*egen + 0.002*holde + 0.002*svare + 0.002*jobbe'),
 (1,
  '0.005*VG + 0.004*spiller + 0.004*  + 0.004*spille + 0.003*møte + 0.002*fortelle + 0.002*vite + 0.002*skje + 0.002*klubbe + 0.002*sesong + 0.002*Solskjær + 0.002*kamp + 0.002*parti + 0.002*snakke + 0.002*dag + 0.002*VM + 0.002*lag + 0.002*holde + 0.002*vinne + 0.002*små'),
 (2,
  '0.005*VG + 0.003*spiller + 0.003*vite + 0.003*skje + 0.003*møte + 0.003*snakke + 0.003*spille + 0.002*fortelle + 0.002*jobb + 0.002*Real_Madrid + 0.002*små + 0.002*jobbe + 0.002*Trump + 0.002*Solskjær + 0.002*svare + 0.002*egen + 0.002*  + 0.002*barn + 0.002*legge + 0.002*finne'),
 (3,
  '0.005*Solskjær + 0.005*spiller + 0.004*spille + 0.003*VG + 0.003*avtale + 0.003*skje + 0.003*jobb + 0.002*  + 0.002*klubbe + 0.002*Manchester_United 

## LDA

In [67]:
ldamodel = LdaModel(corpus=corpus, num_topics=20, id2word=dictionary)
ldamodel.show_topics()


[(1,
  '0.022*"spille" + 0.015*"spiller" + 0.011*"klubbe" + 0.008*"kamp" + 0.008*"VG" + 0.007*"trener" + 0.007*"sesong" + 0.006*"landslag" + 0.006*"lag" + 0.005*"Brann"'),
 (8,
  '0.016*"VG" + 0.008*"skrive" + 0.007*"fortelle" + 0.006*"kjenne" + 0.005*"kjent" + 0.005*"møte" + 0.005*"medium" + 0.005*"barn" + 0.005*"sende" + 0.005*"bilde"'),
 (9,
  '0.020*"Berg" + 0.008*"pave" + 0.007*"muslim" + 0.007*"Thomas" + 0.006*"islam" + 0.005*"jøde" + 0.005*"sjakk" + 0.005*"klasserom" + 0.003*"male" + 0.003*"Sophie_Elise"'),
 (10,
  '0.012*"Trump" + 0.008*"møte" + 0.007*"russisk" + 0.007*" " + 0.005*"angrep" + 0.005*"amerikansk" + 0.004*"president" + 0.004*"VG" + 0.004*"skje" + 0.004*"bruke"'),
 (5,
  '0.011*"vinne" + 0.008*"kjøre" + 0.008*"VG" + 0.008*"VM" + 0.007*"nummer" + 0.007*"met" + 0.006*"finale" + 0.006*"seier" + 0.006*"slå" + 0.006*"jo"'),
 (18,
  '0.022*"barn" + 0.021*"fortelle" + 0.013*"familie" + 0.010*"liv" + 0.008*"gammel" + 0.008*"vite" + 0.008*"dag" + 0.007*"snakke" + 0.007*"mor"

## pyLDAvis

In [176]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.066180 -0.024478       1        1  15.262993
2     -0.140050  0.138752       2        1  15.225776
5     -0.123996  0.063841       3        1  12.392921
6      0.046852 -0.089386       4        1  11.317782
3     -0.001104 -0.018376       5        1  10.878724
8      0.076837 -0.089160       6        1   9.661888
1     -0.045195 -0.010493       7        1   9.093607
4     -0.083401  0.029757       8        1   8.724104
7      0.040426 -0.152278       9        1   6.863863
0      0.295811  0.151821      10        1   0.578346, topic_info=           Term          Freq         Total Category  logprob  loglift
1268      Trump  12877.000000  12877.000000  Default  30.0000  30.0000
389       parti  16698.000000  16698.000000  Default  29.0000  29.0000
10168  Solskjær  10141.000000  10141.000000  Default  28.0000  28.0000
1609       barn  16190.000000  16190.000000  Default  27.0000  27.0000
3534    spiller  10021.000000  10021.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
325       Høyre     64.600334   4786.334961  Topic10  -6.2348   0.8475
415      støtte     66.756042   8975.401367  Topic10  -6.2020   0.2516
337          VG     64.573128  39785.218750  Topic10  -6.2352  -1.2707
327         KrF     61.117802   6308.246582  Topic10  -6.2902   0.5159
2703      Syria     58.599373   2439.264893  Topic10  -6.3323   1.4240

[898 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.101088     
0         2  0.096384     
0         3  0.116368     
0         4  0.114779     
0         5  0.183490     
...     ...       ...  ...
3203      6  0.009416  øke
3203      7  0.011299  øke
3203      8  0.003766  øke
3203      9  0.088277  øke
3203     10  0.000471  øke

[3622 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 3, 6, 7, 4, 9, 2, 5, 8, 1])